In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import optuna
from optuna.pruners import SuccessiveHalvingPruner
from statsmodels.stats.outliers_influence import variance_inflation_factor
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
# чтение данных
df = pd.read_csv('dataset.csv')
df.head()

,"Макс. ПДЗ за Y-1 год, дней","Сред. ПДЗ за Y-1 год, дней","Кол-во просрочек свыше 5-ти дней за Y-1 год, шт.","Общая сумма ПДЗ свыше 5-ти дней за Y-1 год, руб.","Кол-во раз ПДЗ за Y-1 год, шт.",Факт 1,Факт 2,Факт 3,Факт 4,Факт 5,...,"Y-1, Прибыль (убыток) до налогообложения , RUB","Y-4, Прибыль (убыток) от продажи, RUB","Y-3, Прибыль (убыток) от продажи, RUB","Y-2, Прибыль (убыток) от продажи, RUB","Y-1, Прибыль (убыток) от продажи, RUB",Факт просрочки,Просрочка более 30 дней,Просрочка 0-30,"Оценка потенциала контрагента 1, руб.","Оценка потенциала контрагента 2, руб."
0,0,0.0,0,0.000000,0,-10,-10.0,-10.0,-10,-10,...,3.603784e+09,3.280355e+09,6.200120e+09,8.716191e+08,3.658634e+09,1,0,1,-1.0,-1.0
1,0,0.0,0,0.000000,0,-10,-10.0,-10.0,-10,-10,...,8.747516e+07,1.630064e+07,1.109172e+07,5.135732e+07,9.411019e+07,1,0,1,-1.0,-1.0
2,7,5.5,1,132825.299363,2,-10,-10.0,-10.0,-10,-10,...,-6.456439e+08,4.148586e+08,1.611318e+08,-9.298981e+07,-1.207210e+08,1,0,1,-1.0,-1.0
3,0,0.0,0,0.000000,0,-10,-10.0,-10.0,-10,-10,...,3.999298e+09,4.903117e+09,5.186553e+09,7.869977e+09,4.029232e+09,1,0,1,-1.0,-1.0
4,2,2.0,0,0.000000,2,-10,-10.0,-10.0,-10,-10,...,4.960408e+10,2.338912e+10,3.727984e+10,5.307524e+10,5.622122e+10,1,0,1,-1.0,-1.0


In [3]:
df.shape

(853, 135)

In [4]:
print('Факт просрочки')
print(df['Факт просрочки'].value_counts())
print('\nПросрочка 0-30')
print(df['Просрочка 0-30'].value_counts())
print('\nПросрочка более 30 дней')
print(df['Просрочка более 30 дней'].value_counts())

Факт просрочки
1    471
0    382
Name: Факт просрочки, dtype: int64

Просрочка 0-30
1    503
0    350
Name: Просрочка 0-30, dtype: int64

Просрочка более 30 дней
0    696
1    157
Name: Просрочка более 30 дней, dtype: int64


In [5]:
corr_matrix = df.corr()[['Факт просрочки', 
                         'Просрочка 0-30', 
                         'Просрочка более 30 дней']].abs().sort_values(by = ['Факт просрочки', 
                         'Просрочка 0-30', 
                         'Просрочка более 30 дней'], ascending = False).drop(['Факт просрочки', 
                         'Просрочка 0-30', 
                         'Просрочка более 30 дней'])
corr_matrix.style.format("{:.3}").background_gradient(cmap = 'coolwarm')

,Факт просрочки,Просрочка 0-30,Просрочка более 30 дней
Факт 47,0.343,0.653,0.0887
Факт 4,0.339,0.663,0.0919
Факт 8,0.339,0.663,0.0919
Факт 11,0.339,0.663,0.0919
Факт 17,0.339,0.663,0.0919
Факт 18,0.339,0.663,0.0919
Факт 19,0.339,0.663,0.0919
Факт 21,0.339,0.663,0.0919
Факт 22,0.339,0.663,0.0919
Факт 34,0.339,0.663,0.0919


## Исследование
* Проведем исследование подхода к моделированию на таргете "Факт просрочки"

### Выбор модели
* Выберем и исследуем два типа моделей:
    * Логистическая регрессия
    * Градиентный бустинг (catboost)

### Предобработка данных
* Предварительно нормализуем значения фичей для ускорения моделирования (отметим, что это не обязательный шаг, но так процесс схождения к минимуму выполнится быстрее) - требуется для логистической регрессии
* Так как в датасете присутствуют три таргета - для каждого требуется построить отдельную модель

### Подбор параметров
* Подберем паракметры для с помощью оптимизатора optuna
* Отметим, что так как у нас несбалансированная выборка, мы будем использовать стратифицированную кросс-валидацию и метрику roc_auc_score для проверки качества

### Факт просрочки

In [6]:
def objective(trial):
    classifier_name = trial.suggest_categorical('classifier', ['LogisticRegression', 'CatBoostClassifier'])
    
    y = df['Факт просрочки']
    X = df.drop(['Факт просрочки', 'Просрочка более 30 дней', 'Просрочка 0-30'], axis=1)

    if classifier_name == 'LogisticRegression':
        
        # набор гиперпараметров
        lr_c = trial.suggest_loguniform('c', 1e-10, 1e10)
        lr_class_weight = trial.suggest_categorical('class_weight', [None, 'balanced'])
        lr_solver = trial.suggest_categorical('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'])
        
        # классификатор
        classifier_obj = LogisticRegression(max_iter=10000, 
                                            random_state=42,
                                            C=lr_c, 
                                            class_weight=lr_class_weight, 
                                            solver=lr_solver)
        
        # нормализация данных
        scaler = StandardScaler()
        transformer = ColumnTransformer([("st_scaler", 
                                     scaler, 
                                     X.columns)],
                                     remainder="passthrough")
        X = transformer.fit_transform(X)
    else:
        
        # набор гиперпараметров
        cb_l2_leaf_reg = trial.suggest_int('l2_leaf_reg', 1, 9)
        cb_learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
        cb_depth = trial.suggest_int('depth', 3, 10)
        
        # классификатор
        classifier_obj = CatBoostClassifier(depth=cb_depth, 
                                            learning_rate=cb_learning_rate, 
                                            l2_leaf_reg=cb_l2_leaf_reg, 
                                            verbose=0)
        
        X = np.array(X)

    
    # обучение на кросс-валидации
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_score = []
    for train_index, test_index in kf.split(X,y):
        
        #split train/test
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        
        # fit and predict
        clf = classifier_obj.fit(X_train, y_train)
        score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        cv_score.append(score)

    return np.mean(cv_score)

In [7]:
# verbose=0
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [8]:
%%time

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

CPU times: user 4h 28min 56s, sys: 6min 9s, total: 4h 35min 5s
Wall time: 46min 53s


Построим модель, используя лучшие параметры, подобранные оптимизатором:

In [9]:
study.best_params, study.best_value

({'classifier': 'CatBoostClassifier',
  'l2_leaf_reg': 2,
  'learning_rate': 0.006457866785330533,
  'depth': 5},
 0.7629145116685637)

In [10]:
y = df['Факт просрочки']
X = df.drop(['Факт просрочки', 'Просрочка более 30 дней', 'Просрочка 0-30'], axis=1)


if study.best_params['classifier'] == 'LogisticRegression':
         
    # классификатор
    classifier_obj = LogisticRegression(max_iter=10000, 
                                        random_state=42,
                                        C=study.best_params['c'], 
                                        class_weight=study.best_params['class_weight'], 
                                        solver=study.best_params['solver'])
        
    # нормализация данных
    scaler = StandardScaler()
    transformer = ColumnTransformer([("st_scaler", 
                                 scaler, 
                                 X.columns)],
                                 remainder="passthrough")
    X = transformer.fit_transform(X)
else:
        
    # классификатор
    classifier_obj = CatBoostClassifier(depth=study.best_params['depth'], 
                                        learning_rate=study.best_params['learning_rate'],  
                                        l2_leaf_reg=study.best_params['l2_leaf_reg'],
                                        verbose=0)

    X = np.array(X)

    
# обучение на кросс-валидации
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_score = []
i = 1
for train_index, test_index in kf.split(X,y):
    print('{} из KFold {}'.format(i, kf.n_splits))
    #split train/test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]

    # fit and predict
    clf = classifier_obj.fit(X_train, y_train)
    score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
    print('ROC_AUC score:',score)
    i += 1
    cv_score.append(score)

print(f"\nСредний ROC_AUC score: {np.mean(cv_score)}")

1 из KFold 5
ROC_AUC score: 0.7475069252077563
2 из KFold 5
ROC_AUC score: 0.7352859906051395
3 из KFold 5
ROC_AUC score: 0.806023763470572
4 из KFold 5
ROC_AUC score: 0.7820548712206048
5 из KFold 5
ROC_AUC score: 0.7437010078387458

Средний ROC_AUC score: 0.7629145116685637


### VIF score

Вычислим для каждой фичи коэффициент корреляции дисперсии (VIF). Известно, что чем больше увеличивается VIF, тем менее надежными будут результаты модели. В целом, значение VIF выше 10 указывает на высокую корреляцию и является поводом для беспокойства.

Оставим только фичи с коэффициентом VIF менее 10 и снова подберем гиперпараметры с помощью оптимизатора.

In [11]:
vif_data = pd.DataFrame()
tmp = df.drop(['Факт просрочки', 'Просрочка более 30 дней', 'Просрочка 0-30'], axis=1)
vif_data['feature'] = tmp.columns

vif_data['vif'] = [variance_inflation_factor(tmp.values, i) for i in range(len(tmp.columns))]
features_vif = vif_data[vif_data['vif'] < 10]['feature'].values
vif_data.head()

,feature,vif
0,"Макс. ПДЗ за Y-1 год, дней",3.351972
1,"Сред. ПДЗ за Y-1 год, дней",2.616495
2,"Кол-во просрочек свыше 5-ти дней за Y-1 год, шт.",4.984048
3,"Общая сумма ПДЗ свыше 5-ти дней за Y-1 год, руб.",3.631964
4,"Кол-во раз ПДЗ за Y-1 год, шт.",4.363830


In [12]:
def objective_vif(trial):
    classifier_name = trial.suggest_categorical('classifier', ['LogisticRegression', 'CatBoostClassifier'])
    
    y = df['Факт просрочки']
    X = df.drop(['Факт просрочки', 'Просрочка более 30 дней', 'Просрочка 0-30'], axis=1)
    X = X[features_vif]

    if classifier_name == 'LogisticRegression':
        
        # набор гиперпараметров
        lr_c = trial.suggest_loguniform('c', 1e-10, 1e10)
        lr_class_weight = trial.suggest_categorical('class_weight', [None, 'balanced'])
        lr_solver = trial.suggest_categorical('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'])
        
        # классификатор
        classifier_obj = LogisticRegression(max_iter=10000, 
                                            random_state=42,
                                            C=lr_c, 
                                            class_weight=lr_class_weight, 
                                            solver=lr_solver)
        
        # нормализация данных
        scaler = StandardScaler()
        transformer = ColumnTransformer([("st_scaler", 
                                     scaler, 
                                     X.columns)],
                                     remainder="passthrough")
        X = transformer.fit_transform(X)
    else:
        
        # набор гиперпараметров
        cb_l2_leaf_reg = trial.suggest_int('l2_leaf_reg', 1, 9)
        cb_learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
        cb_depth = trial.suggest_int('depth', 3, 10)
        
        # классификатор
        classifier_obj = CatBoostClassifier(depth=cb_depth, 
                                            learning_rate=cb_learning_rate, 
                                            l2_leaf_reg=cb_l2_leaf_reg, 
                                            verbose=0)
        
        X = np.array(X)

    
    # обучение на кросс-валидации
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_score = []
    for train_index, test_index in kf.split(X,y):
        
        #split train/test
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        
        # fit and predict
        clf = classifier_obj.fit(X_train, y_train)
        score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        cv_score.append(score)

    return np.mean(cv_score)

In [13]:
%%time

study_vif = optuna.create_study(direction='maximize')
study_vif.optimize(objective_vif, n_trials=100)

CPU times: user 47min 27s, sys: 3min 18s, total: 50min 45s
Wall time: 10min 24s


Построим модель, используя лучшие параметры, подобранные оптимизатором:

In [14]:
study_vif.best_params, study_vif.best_value

({'classifier': 'CatBoostClassifier',
  'l2_leaf_reg': 8,
  'learning_rate': 0.007206028650368354,
  'depth': 4},
 0.7825453592496971)

In [15]:
y = df['Факт просрочки']
X = df.drop(['Факт просрочки', 'Просрочка более 30 дней', 'Просрочка 0-30'], axis=1)
X = X[features_vif]

if study_vif.best_params['classifier'] == 'LogisticRegression':
         
    # классификатор
    classifier_obj = LogisticRegression(max_iter=10000, 
                                        random_state=42,
                                        C=study_vif.best_params['c'], 
                                        class_weight=study_vif.best_params['class_weight'], 
                                        solver=study_vif.best_params['solver'])
        
    # нормализация данных
    scaler = StandardScaler()
    transformer = ColumnTransformer([("st_scaler", 
                                 scaler, 
                                 X.columns)],
                                 remainder="passthrough")
    X = transformer.fit_transform(X)
else:
        
    # классификатор
    classifier_obj = CatBoostClassifier(depth=study_vif.best_params['depth'], 
                                        learning_rate=study_vif.best_params['learning_rate'],  
                                        l2_leaf_reg=study_vif.best_params['l2_leaf_reg'],
                                        verbose=0)

    X = np.array(X)

    
# обучение на кросс-валидации
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_score = []
i = 1
for train_index, test_index in kf.split(X,y):
    print('{} из KFold {}'.format(i, kf.n_splits))
    #split train/test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]

    # fit and predict
    clf = classifier_obj.fit(X_train, y_train)
    score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
    print('ROC_AUC score:',score)
    i += 1
    cv_score.append(score)

print(f"\nСредний ROC_AUC score: {np.mean(cv_score)}")

1 из KFold 5
ROC_AUC score: 0.771606648199446
2 из KFold 5
ROC_AUC score: 0.7702403978999725
3 из KFold 5
ROC_AUC score: 0.8131389886709036
4 из KFold 5
ROC_AUC score: 0.7982222844344905
5 из KFold 5
ROC_AUC score: 0.7595184770436731

Средний ROC_AUC score: 0.7825453592496971


### ASHA

Один из методов многорукого бандита для подобора гиперпараметров называется ASHA (асинхронное последовательное деление пополам). Общая идея такова:

* запустить кучу конфигураций параметров в течение некоторого времени
* обрезать (половину) наименее перспективных запусков 
* запустите кучу конфигураций параметров еще на некоторое время
* обрезать (половину) наименее перспективных запусков 
* остановить, когда останется только одна конфигурация

Таким образом, поиск может сосредоточиться на более многообещающих запусках.

In [16]:
def objective_vif_pruner(trial):
    
    y = df['Факт просрочки']
    X = df.drop(['Факт просрочки', 'Просрочка более 30 дней', 'Просрочка 0-30'], axis=1)
    X = X[features_vif]
 
    # набор гиперпараметров
    cb_l2_leaf_reg = trial.suggest_int('l2_leaf_reg', 1, 9)
    cb_learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    cb_depth = trial.suggest_int('depth', 3, 10)

    # классификатор
    classifier_obj = CatBoostClassifier(depth=cb_depth, 
                                        learning_rate=cb_learning_rate, 
                                        l2_leaf_reg=cb_l2_leaf_reg, 
                                        verbose=0)

    X = np.array(X)

    
    # обучение на кросс-валидации
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_score = []
    for train_index, test_index in kf.split(X,y):
        
        #split train/test
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        
        # fit and predict
        clf = classifier_obj.fit(X_train, y_train)
        score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        cv_score.append(score)

    return np.mean(cv_score)

In [17]:
%%time

study_vif_pruner = optuna.create_study(direction='maximize', pruner=SuccessiveHalvingPruner)
study_vif_pruner.optimize(objective_vif_pruner, n_trials=100)

CPU times: user 1h 11min 33s, sys: 4min 50s, total: 1h 16min 24s
Wall time: 15min 42s


Построим модель, используя лучшие параметры, подобранные оптимизатором:

In [18]:
study_vif_pruner.best_params, study_vif_pruner.best_value

({'l2_leaf_reg': 9, 'learning_rate': 0.006606180680317283, 'depth': 6},
 0.7808419533257454)

In [19]:
y = df['Факт просрочки']
X = df.drop(['Факт просрочки', 'Просрочка более 30 дней', 'Просрочка 0-30'], axis=1)
X = X[features_vif]

        
# классификатор
classifier_obj = CatBoostClassifier(depth=study_vif_pruner.best_params['depth'], 
                                    learning_rate=study_vif_pruner.best_params['learning_rate'],  
                                    l2_leaf_reg=study_vif_pruner.best_params['l2_leaf_reg'],
                                    verbose=0)

X = np.array(X)

    
# обучение на кросс-валидации
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_score = []
i = 1
for train_index, test_index in kf.split(X,y):
    print('{} из KFold {}'.format(i, kf.n_splits))
    #split train/test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]

    # fit and predict
    clf = classifier_obj.fit(X_train, y_train)
    score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
    print('ROC_AUC score:',score)
    i += 1
    cv_score.append(score)

print(f"\nСредний ROC_AUC score: {np.mean(cv_score)}")

1 из KFold 5
ROC_AUC score: 0.7678670360110803
2 из KFold 5
ROC_AUC score: 0.7753523072672011
3 из KFold 5
ROC_AUC score: 0.7987703785576127
4 из KFold 5
ROC_AUC score: 0.7989221724524077
5 из KFold 5
ROC_AUC score: 0.7632978723404255

Средний ROC_AUC score: 0.7808419533257454


### Просрочка 0-30
* Проделаем последнюю итерацию предыдущего пайплайна для таргета "Просрочка 0-30"

In [20]:
def objective_vif_pruner_30(trial):
    
    y = df['Просрочка 0-30']
    X = df.drop(['Факт просрочки', 'Просрочка более 30 дней', 'Просрочка 0-30'], axis=1)
    X = X[features_vif]
 
    # набор гиперпараметров
    cb_l2_leaf_reg = trial.suggest_int('l2_leaf_reg', 1, 9)
    cb_learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    cb_depth = trial.suggest_int('depth', 3, 10)

    # классификатор
    classifier_obj = CatBoostClassifier(depth=cb_depth, 
                                        learning_rate=cb_learning_rate, 
                                        l2_leaf_reg=cb_l2_leaf_reg, 
                                        verbose=0)

    X = np.array(X)

    
    # обучение на кросс-валидации
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_score = []
    for train_index, test_index in kf.split(X,y):
        
        #split train/test
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        
        # fit and predict
        clf = classifier_obj.fit(X_train, y_train)
        score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        cv_score.append(score)

    return np.mean(cv_score)

In [21]:
%%time

study_30 = optuna.create_study(direction='maximize', pruner=SuccessiveHalvingPruner)
study_30.optimize(objective_vif_pruner_30, n_trials=100)

CPU times: user 47min 28s, sys: 3min 36s, total: 51min 4s
Wall time: 10min 19s


In [22]:
study_30.best_params, study_30.best_value

({'l2_leaf_reg': 1, 'learning_rate': 0.0016288374924997603, 'depth': 4},
 0.893242008486563)

In [23]:
y = df['Просрочка 0-30']
X = df.drop(['Факт просрочки', 'Просрочка более 30 дней', 'Просрочка 0-30'], axis=1)
X = X[features_vif]

        
# классификатор
classifier_obj = CatBoostClassifier(depth=study_30.best_params['depth'], 
                                    learning_rate=study_30.best_params['learning_rate'],  
                                    l2_leaf_reg=study_30.best_params['l2_leaf_reg'],
                                    verbose=0)

X = np.array(X)

    
# обучение на кросс-валидации
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_score = []
i = 1
for train_index, test_index in kf.split(X,y):
    print('{} из KFold {}'.format(i, kf.n_splits))
    #split train/test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]

    # fit and predict
    clf = classifier_obj.fit(X_train, y_train)
    score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
    print('ROC_AUC score:',score)
    i += 1
    cv_score.append(score)

print(f"\nСредний ROC_AUC score: {np.mean(cv_score)}")

1 из KFold 5
ROC_AUC score: 0.9302687411598303
2 из KFold 5
ROC_AUC score: 0.8816124469589816
3 из KFold 5
ROC_AUC score: 0.8956859971711457
4 из KFold 5
ROC_AUC score: 0.8780714285714285
5 из KFold 5
ROC_AUC score: 0.8805714285714286

Средний ROC_AUC score: 0.893242008486563


### Просрочка более 30 дней

In [24]:
def objective_vif_pruner_more_30(trial):
    
    y = df['Просрочка более 30 дней']
    X = df.drop(['Факт просрочки', 'Просрочка более 30 дней', 'Просрочка 0-30'], axis=1)
    X = X[features_vif]
 
    # набор гиперпараметров
    cb_l2_leaf_reg = trial.suggest_int('l2_leaf_reg', 1, 9)
    cb_learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    cb_depth = trial.suggest_int('depth', 3, 10)

    # классификатор
    classifier_obj = CatBoostClassifier(depth=cb_depth, 
                                        learning_rate=cb_learning_rate, 
                                        l2_leaf_reg=cb_l2_leaf_reg, 
                                        verbose=0)

    X = np.array(X)

    
    # обучение на кросс-валидации
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_score = []
    for train_index, test_index in kf.split(X,y):
        
        #split train/test
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        
        # fit and predict
        clf = classifier_obj.fit(X_train, y_train)
        score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        cv_score.append(score)

    return np.mean(cv_score)

In [25]:
%%time

study_more_30 = optuna.create_study(direction='maximize', pruner=SuccessiveHalvingPruner)
study_more_30.optimize(objective_vif_pruner_more_30, n_trials=100)

CPU times: user 1h 40min 29s, sys: 6min 29s, total: 1h 46min 59s
Wall time: 23min 7s


In [26]:
study_more_30.best_params, study_more_30.best_value

({'l2_leaf_reg': 2, 'learning_rate': 0.0030122905328414528, 'depth': 8},
 0.7960159384013525)

In [27]:
y = df['Просрочка более 30 дней']
X = df.drop(['Факт просрочки', 'Просрочка более 30 дней', 'Просрочка 0-30'], axis=1)
X = X[features_vif]

        
# классификатор
classifier_obj = CatBoostClassifier(depth=study_more_30.best_params['depth'], 
                                    learning_rate=study_more_30.best_params['learning_rate'],  
                                    l2_leaf_reg=study_more_30.best_params['l2_leaf_reg'],
                                    verbose=0)

X = np.array(X)

    
# обучение на кросс-валидации
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_score = []
i = 1
for train_index, test_index in kf.split(X,y):
    print('{} из KFold {}'.format(i, kf.n_splits))
    #split train/test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]

    # fit and predict
    clf = classifier_obj.fit(X_train, y_train)
    score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
    print('ROC_AUC score:',score)
    i += 1
    cv_score.append(score)

print(f"\nСредний ROC_AUC score: {np.mean(cv_score)}")

1 из KFold 5
ROC_AUC score: 0.838479262672811
2 из KFold 5
ROC_AUC score: 0.8412769784172661
3 из KFold 5
ROC_AUC score: 0.7079586330935252
4 из KFold 5
ROC_AUC score: 0.800533766535159
5 из KFold 5
ROC_AUC score: 0.7918310512880018

Средний ROC_AUC score: 0.7960159384013525
